In [ ]:
import pandas as pd
import os 
import numpy as np

In [2]:
# define the base path to thumb drive (input path)
base_path = '/Volumes/Untitled/MeterDataTest'
#base_path = '/Desktop/MeterDataTest'

#check if base path exists
if not os.path.exists(base_path):
    print(f"Error: Path {base_path} does not exist")
    exit()

In [3]:
# list of dataframes of each meter
meter_dfs = []

# list to store all dataframes of each meter
all_data = []

#list to store all dataframes with 3_phase_watt_total column for later use
save_data = []

In [ ]:
# iterate through the subfolders in the MeterDataTest folder
for subfolder in os.listdir(base_path):
    # create path for each subfolder
    folder_path = os.path.join(base_path, subfolder)

    if not os.path.isdir(folder_path):
        continue
    
    # get the name of the meter from the subfolder name, make lowercase
    meter_name = subfolder.lower().replace(" ", "_").replace("_mtr", "")
    #print(meter_name)

    # list of csv file paths in subfolder
    # addition with the 'and not' is to make sure to ignore the hidden ._ files
    csv_paths = [os.path.join(folder_path, f) 
                 for f in os.listdir(folder_path) 
                 if f.endswith('.csv')
                 and not f.startswith("._")
                 and not f.startswith(".")]

    # empty meter's dataframe list
    meter_dfs = []
    
    # convert each csv to a df, fix columns and add df to df list
    for csv in csv_paths:
        temp_df = pd.read_csv(csv, encoding="utf-8")

        temp_df.columns = temp_df.columns.str.strip().str.lower().str.replace(" ", "_")
        
        # rename columns if they exist
        # some meters have have the different label but they are synonymous
        if '3_phase_positive_real_energy_used' in temp_df.columns:
            temp_df.rename(columns={
                '3_phase_positive_real_energy_used': 'total_watt_hour',
                '3_phase_real_power':'3_phase_watt_total'
            }, inplace=True)

        # error in scripts, total_watt_hour is actually total kwh
        temp_df.rename(columns={'total_watt_hour': 'kwh'}, inplace=True)
        
        # reorder the columns
        temp_df = temp_df[['datetime', 'kwh', '3_phase_watt_total']]
        temp_df.insert(1, 'meter_name', meter_name)

        meter_dfs.append(temp_df)

    # combine all csvs for this meter into one dataframe
    df = pd.concat(meter_dfs, ignore_index=True)
    
    # save the dataframe with all columns to list
    save_data.append(df.copy())
    df.drop('3_phase_watt_total', axis=1, inplace=True)

    # convert datetime column to a datetime type
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')

    # create column that contains the closest interval for each timestamp (contains ymd hms, using timedelta)
    #df['interval_15min'] = pd.to_datetime(df['datetime'].dt.round('15min'), format='%Y-%m-%d %H:%M:%S')
    df['interval_15min'] = df['datetime'].dt.round('15min')
    
    # create column that contains the offset in seconds from the closest interval for each timestamp
    # - is if its before it and + is if its after
    df['interval_offset'] = (df['datetime'] - df['interval_15min']).dt.total_seconds()

    # create new column with true if an exact interval and false if not
    df['is_exact'] = df['datetime'].eq(df['interval_15min'])
    
    df['interpolated'] = False
    
    interpolated_rows = []

    # interval = the 15min bucket val, group = all rows in that bucket
    for interval, group in df.groupby('interval_15min'):
        # only select rows in group with is_exact == True
        exact = group[group['is_exact']]

        if exact.empty:
            before = group[group['interval_offset'] <= 0]
            after = group[group['interval_offset'] >= 0]

            # check if there are empties
            if not before.empty and not after.empty:
                # grab the closest data to the interval
                time_before = before.iloc[-1]
                time_after = after.iloc[0]
                
                # calculate the estimated kwh
                # get the slope to 4 decimal places
                reading_diff = time_after['kwh'] - time_before['kwh']

                if reading_diff == 0:
                    estimated_twh = time_before['kwh']
                else:
                    time_diff = (time_after['datetime'] - time_before['datetime']).total_seconds()
                    slope = round(reading_diff / time_diff, 4)
                    sec_before_interval = (interval - time_before['datetime']).total_seconds()
                    estimated_twh = time_before['kwh'] + (slope * sec_before_interval)

                # create interpolated row
                new_row = time_before.copy()
                new_row['datetime'] = interval
                new_row['kwh'] = estimated_twh
                new_row['interval_offset'] = 0
                new_row['is_exact'] = True
                new_row['interpolated'] = True
                
                # add new interpolated row to list
                interpolated_rows.append(new_row)

    # combine interpolated data with dataframe
    if interpolated_rows:
        
        df = pd.concat([df, pd.DataFrame(interpolated_rows)], ignore_index=True)

    df = df.drop(columns=['interval_15min', 'interval_offset'])

    # resort the data to be in order of datetime
    df = df.sort_values(by='datetime').reset_index(drop=True)
    
    all_data.append(df)

In [ ]:
# dataframe for all data
combined_data = pd.DataFrame()
save_data_col = pd.DataFrame()

In [ ]:
# create with all columns to save for later
save_data_col = pd.concat(save_data, ignore_index=True)

In [ ]:
# combine all dataframes in list to one dataframe
combined_data = pd.concat(all_data, ignore_index=True)

In [ ]:
# check there is only one instance
print(combined_data[(combined_data['meter_name'] == 'admin_serv_1_mtr') &
    (combined_data['datetime'] == pd.to_datetime('2025-07-23 09:40:50'))])

In [ ]:
# check for if there exists duplicates
duplicate_data = combined_data[combined_data.duplicated(keep=False)]
print(duplicate_data)

In [ ]:
print(combined_data.head(15))

In [ ]:
# convert dataframe with all collumns to csv to save for later
save_data_col.to_csv('save_data_col.csv', index=False)

In [ ]:
# convert dataframe to csv
combined_data.to_csv('interpolated_meter_data.csv', index=False)

In [ ]:
# list of meters and first timestamp in dataset
df = pd.read_csv('interpolated_meter_data.csv', encoding="utf-8")
print(df['meter_name'].unique())
print(df.head(1))

In [ ]:
# sample of interpolated data for admin_serv_1 2025-09-10
df = pd.read_csv('interpolated_meter_data.csv', encoding="utf-8")
df['datetime'] = pd.to_datetime(df['datetime'])
date = pd.to_datetime('2025-09-10').date()
sample_data = df[(df['datetime'].dt.date == date) & (df['meter_name'] == 'admin_serv_1_mtr')]
sample_data.to_csv('sample_data.csv', index=False)

In [ ]:
# sample of interpolated data for biomedical_science_main_a_mtr 2025-09-10
df = pd.read_csv('interpolated_meter_data.csv', encoding='utf-8')

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])
date = pd.to_datetime('2025-09-10').date()
sample_data = df[(df['datetime'].dt.date == date) 
    & (df['meter_name'] == 'biomedical_science_main_a_mtr') 
    & ((df['interpolated'] == True) | ((df['datetime'].dt.minute % 15 == 0) & (df['datetime'].dt.second == 0)))
    ].drop_duplicates(subset=['datetime', 'meter_name'])

In [ ]:
sample_data.to_csv('sample_data.csv', index=False)

In [6]:
# get just interpolated meter data from all meters on days sept 07-09 2025
df = pd.read_csv('interpolated_meter_data.csv', encoding='utf-8')
df['datetime'] = pd.to_datetime(df['datetime'])

In [10]:
df = df[df['datetime'].dt.date.isin([
    pd.to_datetime('2025-09-07').date(),
    pd.to_datetime('2025-09-08').date(),
    pd.to_datetime('2025-09-09').date()])]
df = df[df['is_exact'] & True]

In [11]:
print(df)

                   datetime         meter_name           kwh  is_exact  \
47585   2025-09-07 00:00:00       admin_serv_1  1.404952e+06      True   
47601   2025-09-07 00:15:00       admin_serv_1  1.404953e+06      True   
47617   2025-09-07 00:30:00       admin_serv_1  1.404955e+06      True   
47633   2025-09-07 00:45:00       admin_serv_1  1.404957e+06      True   
47649   2025-09-07 01:00:00       admin_serv_1  1.404959e+06      True   
...                     ...                ...           ...       ...   
8722916 2025-09-09 22:45:00  wist_annex_1_main  6.486340e+05      True   
8722932 2025-09-09 23:00:00  wist_annex_1_main  6.486370e+05      True   
8722948 2025-09-09 23:15:00  wist_annex_1_main  6.486390e+05      True   
8722964 2025-09-09 23:30:00  wist_annex_1_main  6.486400e+05      True   
8722980 2025-09-09 23:45:00  wist_annex_1_main  6.486420e+05      True   

         interpolated  
47585            True  
47601            True  
47617            True  
47633          

In [12]:
df.to_csv('sept07-09_kwh.csv', index=False)